# Producer multiple topics

In [16]:
from kafka import KafkaProducer
from kafka.admin import KafkaAdminClient, NewTopic
import datetime as dt
import pprint
import kafka
import time
import json
import os

# creds, config = read_files()
# subreddit = config["subreddit"]
# post_type = config["post_type"]
# debug = config["debug"]
# kafka_host = config["kafka_host"]
# spark_host = config["spark_host"]
# aws_client = creds["aws-client"]
# aws_secret = creds["aws-secret"]
try:
    from reddit.reddit_producer import *
    from reddit.reddit_streaming import read_files

except Exception as e:
    print(e)

print("imported modules")

imported modules


In [17]:
os.environ["subreddit"] = "AsiansGoneWild"
os.environ["AWS_ACCESS_KEY_ID"] = ""
os.environ["AWS_SECRET_ACCESS_KEY"] = ""
print("set environment variables.")

set environment variables.


In [18]:
subreddit = os.environ["subreddit"]
aws_key = os.environ["AWS_ACCESS_KEY_ID"]
aws_secret = os.environ["AWS_SECRET_ACCESS_KEY"]
print("imported variables from os environment.")

imported variables from os environment.


In [19]:
params = {}
params["topic"] = ["reddit_{}".format(subreddit)]
pp.pprint(params)

{'topic': ['reddit_AsiansGoneWild']}


## Connect to kafka broker

In [20]:
def get_bearer():
    return(None)

def my_serializer():
    return(None)

In [21]:
try:
    kafka_host = "xanaxprincess.asuscomm.com"
    broker = ["{}:9092".format(kafka_host)]
    # topic = "reddit_" + subreddit

    producer = KafkaProducer(
                bootstrap_servers=broker,
                value_serializer=my_serializer
            )
    
except kafka.errors.NoBrokersAvailable:
    print("no kafka broker available (error likely to repeat infinitely until resolved)")

header = get_bearer()

no kafka broker available (error likely to repeat infinitely until resolved)


## Initial read from api and send to kafka

In [25]:
token_list = []

for i, s in enumerate(subreddit):
    my_response = get_subreddit(s, 1, post_type, "", header)
    my_data, after_token = subset_response(my_response)
    token_list.append(after_token)
    # with open("sample_response.json", "w") as f:
    #     json.dump(my_data, f, indent = 1)

    if after_token is not None:
        producer.send(params["topic"][i], my_data)

    if debug:
        print("subreddit: {}, post datetime: {}, post title: {}".format(s, dt.datetime.fromtimestamp(my_data["created"]), my_data["title"]))

params["token"] = token_list
print("-------------------------------------------")
        

subreddit: technology, post datetime: 2022-06-16 22:26:07, post title: The Privatized Internet Has Failed Us: The early promises about the utopia that the internet would bring us have proven wrong. The internet can never deliver on all it’s capable of when it’s run for profit — we need a publicly owned internet.
subreddit: ProgrammerHumor, post datetime: 2022-06-16 22:02:26, post title: Based on a true story
subreddit: news, post datetime: 2022-06-16 22:11:46, post title: Anti-vaccine doctor sentenced to prison for Capitol riot
subreddit: worldnews, post datetime: 2022-06-16 22:10:34, post title: Russia needs economic 'perestroika' to end reliance on energy exports - cenbank chief


In [26]:
pp.pprint(params["token"])

['t3_vdxpv7', 't3_vdx7ra', 't3_vdxesk', 't3_vdxdwd']


In [27]:
while True:
    token_list = []
    for i, s in enumerate(subreddit):
        after_token = params["token"][i]
        try:
            next_response = get_subreddit(s, 1, post_type, after_token, header)
            my_data, after_token = subset_response(next_response)
            token_list.append(after_token)

            ## weird bug where it hits the api too fast(?) and no after token is returned
            ## this passes None, which gives the current post & correct access token
            if after_token is not None:
                producer.send(params["topic"][i], my_data)

                if debug:
                    print("subreddit: {}, post datetime: {}, post title: {}".format(s, dt.datetime.fromtimestamp(my_data["created"]), my_data["title"]))
                
            time.sleep(1)

        except json.decoder.JSONDecodeError:
            # when the bearer token expires (after 24 hrs), we do not receive a response
            print("bearer token expired, reauthenticating...")
            header = get_bearer()

            next_response = get_subreddit(s, 1, post_type, after_token, header)
            my_data, after_token = subset_response(next_response)
            token_list.append(after_token)

            if after_token is not None:
                producer.send(params["topic"][i], my_data)

                if debug:
                    print("subreddit: {}, post datetime: {}, post title: {}".format(s, dt.datetime.fromtimestamp(my_data["created"]), my_data["title"]))

            time.sleep(1)
            pass

        except IndexError:
            # this means empty response is returned, take a nap
            # time.sleep(120)
            print("no more data for subreddit: {}.".format(s))
            token_list.append(params["token"][i])
            pass

        except Exception as e:
            # catch all for api exceptions (SSL errors, ConnectionError, etc)
            print(e)
            token_list.append(params["token"][i])
            pass
            # time.sleep(150)

    params["token"] = token_list
    print("-------------------------------------------")
    time.sleep(120)

no more data for subreddit: technology.
no more data for subreddit: ProgrammerHumor.
no more data for subreddit: news.
no more data for subreddit: worldnews.
no more data for subreddit: technology.
no more data for subreddit: ProgrammerHumor.
no more data for subreddit: news.
no more data for subreddit: worldnews.


KeyboardInterrupt: 